In [1]:
# Cambiar el directorio de trabajo al lugar donde estén los datos
import os

# Nuevo directorio
data_directory = 'C:\\Users\\Nicolás Rivera\\OneDrive\\Escritorio\\Procolombia\\Prueba Final\\Datos'
os.chdir(data_directory)

# El sigueinte es el directorio donde se realiza la prueba
os.getcwd() 

'C:\\Users\\Nicolás Rivera\\OneDrive\\Escritorio\\Procolombia\\Prueba Final\\Datos'

In [2]:
## Cargue de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tqdm as notebook_tqdm

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression

# Modelo phophet (pip install fbprophet)
from prophet import Prophet

# Modelo XGBoost (pip install xgboost)
from xgboost import XGBRegressor

# Importar librería para selección de lags (pip install pmdarima)
from pmdarima import auto_arima

## Param
pd.set_option('display.float_format', '{:.2f}'.format)

C:\ProgramData\Anaconda3\envs\mientorno\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Importar base de datos
municipios = pd.read_excel('Tablas Correlativas.xlsx', sheet_name=0)
municipios.head()

,CODIGO_MUNICIPIO,NOMBRE_DEPTO,PROVINCIA,NOMBRE_MPIO
0,'91263,AMAZONAS,AMAZONAS,EL ENCANTO
1,'91405,AMAZONAS,AMAZONAS,LA CHORRERA
2,'91407,AMAZONAS,AMAZONAS,LA PEDRERA
3,'91430,AMAZONAS,AMAZONAS,LA VICTORIA
4,'91001,AMAZONAS,AMAZONAS,LETICIA


In [4]:
country = pd.read_excel('Tablas Correlativas.xlsx', sheet_name=1)
country.head()

,Código ISO país,País
0,ABW,Aruba
1,AFG,Afganistán
2,AGO,Angola
3,AIA,Anguila
4,ALB,Albania


In [5]:
motivo = pd.read_excel('Tablas Correlativas.xlsx', sheet_name=2)
motivo.head()

,Código Motivo de viaje,Motivo de viaje
0,1,"Vacaciones, recreo y ocio"
1,2,Negocios y motivos profesionales
2,3,Eventos
3,4,Educación y formación
4,5,Tránsito


In [6]:
turismo = pd.read_csv("datos_turistas.txt", sep="|")
turismo.head()

,Código ISO país,Código Municipio,Código Motivo de viaje,Género,Mes,Año,Cantidad turistas
0,AFG,'05059,1,F,6,2015,1
1,AFG,'11001,1,F,6,2015,3
2,AFG,'11001,1,M,2,2015,1
3,AFG,'11001,1,M,6,2015,1
4,AFG,'11001,2,M,4,2015,2


In [7]:
municipios = municipios.rename({'CODIGO_MUNICIPIO': 'Código Municipio'}, axis='columns')

In [8]:
df = pd.merge(turismo,municipios,on='Código Municipio',how='left')

In [9]:
df = pd.merge(df,country,on='Código ISO país',how='left')

In [10]:
df = pd.merge(df, motivo, on = 'Código Motivo de viaje', how = 'left')

In [11]:
df.head()

,Código ISO país,Código Municipio,Código Motivo de viaje,Género,Mes,Año,Cantidad turistas,NOMBRE_DEPTO,PROVINCIA,NOMBRE_MPIO,País,Motivo de viaje
0,AFG,'05059,1,F,6,2015,1,ANTIOQUIA,OCCIDENTE,ARMENIA,Afganistán,"Vacaciones, recreo y ocio"
1,AFG,'11001,1,F,6,2015,3,BOGOTÁ D.C.,BOGOTA,BOGOTA D.C.,Afganistán,"Vacaciones, recreo y ocio"
2,AFG,'11001,1,M,2,2015,1,BOGOTÁ D.C.,BOGOTA,BOGOTA D.C.,Afganistán,"Vacaciones, recreo y ocio"
3,AFG,'11001,1,M,6,2015,1,BOGOTÁ D.C.,BOGOTA,BOGOTA D.C.,Afganistán,"Vacaciones, recreo y ocio"
4,AFG,'11001,2,M,4,2015,2,BOGOTÁ D.C.,BOGOTA,BOGOTA D.C.,Afganistán,Negocios y motivos profesionales


In [12]:
## df.to_excel('Turismo - Unión.xlsx', index=True
df.to_csv("Turismo - Unión.csv", index=False)

In [14]:
df.shape

(790586, 12)